In [1]:
import tweepy #using version 3.7.0
import json
import pandas as pd

In [3]:
#Variables that contains the user credentials to access Twitter API 
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [32]:
#creating the list of lists which will eventually become the dataframe
ids = []
full_text = []
created_at = []
label=[]

In [42]:
emotions = {
    'depression': "#depression",
    'anxiety': "#anxiety",
    'mentalhealth': "#mentalhealth"
    }

In [43]:
#https://stackoverflow.com/questions/22469713/managing-tweepy-api-search
def get_tweets(keywords,date_until, num_of_tweets ):
    searched_tweets = []
    last_id = -1
    
    while len(searched_tweets) < num_of_tweets:
        try:
            new_tweets = api.search(q=keywords, count=100, max_id=str(last_id-1), tweet_mode='extended', lang='en', until=date_until)
            if not new_tweets:
                print("No tweets found in this call to api_search")
                break
            for tweet in new_tweets:
                if 'retweeted_status' not in tweet._json:
                    searched_tweets.append(tweet)
            last_id=new_tweets[-1].id
        except tweepy.TweepError as e:
            break
    return searched_tweets

In [44]:
#this cell does the actual scraping. Isolating it so that we don't waste our tweet cap while doing development
searched_tweets = {}
num_of_tweets = 15000
date_until = '2021-12-9'
for emotion, keywords in emotions.items():
    searched_tweets[emotion] = get_tweets(keywords,date_until, num_of_tweets)

No tweets found in this call to api_search


In [45]:
for emotion, tweets in searched_tweets.items():
    print(emotion)
    print(len(tweets))

depression
5582


In [46]:
for emotion, tweets in searched_tweets.items(): #iterating through the emotions
    for tweet in tweets: #iterating through the tweets for each emotion
        ids.append(tweet._json['id_str'])
        created_at.append(tweet._json['created_at'])
        full_text.append(tweet._json['full_text'])
        label.append(1)

In [47]:
dictionary = {
    'id': ids,
    'text': full_text,
    'created_at': created_at,
    'label': label
}
df = pd.DataFrame(dictionary)

In [48]:
df.shape

(11504, 4)

In [49]:
df.head()

,id,text,created_at,label
0,1468731226739646470,😔FOOD AND MOOD. \n\nThe connection between our...,Wed Dec 08 23:56:26 +0000 2021,1
1,1468730364155813888,Online treatment for #anxiety. Effective onlin...,Wed Dec 08 23:53:01 +0000 2021,1
2,1468729608849829896,Speak to a Psychotherapist via Skype. Online M...,Wed Dec 08 23:50:01 +0000 2021,1
3,1468729303172931588,#Mentalhealth #Depression #Anxiety #Bipolar #C...,Wed Dec 08 23:48:48 +0000 2021,1
4,1468729104145039366,Speak to an online therapist for the treatment...,Wed Dec 08 23:48:00 +0000 2021,1


In [50]:
df.to_excel('tweets.xlsx')